In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
from sklearn.feature_extraction.text import CountVectorizer
from annoy import AnnoyIndex
import pickle
import re

In [3]:
df = pd.read_csv('main_data.csv')

Датасет был подготовлен в другом ноутбуке. Обычная чистка текстов и других данных. 
Плюс очищение 'annotation'от шума с помощью TFIDF.feature_extraction()

In [8]:
df['annotation'][1112]

'мир мир который давно сотрясали войны могущественных магов желавших обрести божественное величие власть мир который поднимается руин время надежд покой процветание прекрасное время жить спокойно счастливо могут мятежные души обрести таком мире покой сокровища древних магов бесценные творения неоценимые знания терзает души юнцов почтенных мужей возможность вмиг разбогатеть конца дней жить роскоши многих толкает стезю охотника сокровищами наш герой устоял искусом влился ряды этих безрассудно смелых людей пусть волнует богатство стремится знаниям это меняет главного жизнь охотника сокровищами древних магов очень коротка выбор сделан назад пути'

In [9]:
df['full_text'][1112]

'магов сокровищами охотника покой древних обрести души неоценимые искусом почтенных мир влился желавших устоял стезю'

In [3]:
genres = set()
for i in range(len(df)):
    genres = genres.union(set(df.genres_x[i].split()))

In [4]:
genres

{'18',
 '19',
 '20',
 'None',
 'action',
 'adv',
 'adventure',
 'alternative',
 'anecdote',
 'animal',
 'ant',
 'antique',
 'aphorisms',
 'architecture',
 'art',
 'astrology',
 'auto',
 'banking',
 'behavior',
 'biography',
 'biology',
 'book',
 'budda',
 'business',
 'chem',
 'child',
 'children',
 'childs',
 'christianity',
 'cine',
 'city',
 'classic',
 'classical',
 'classics',
 'collecting',
 'comedy',
 'comp',
 'computers',
 'contemporary',
 'cooking',
 'cosmos',
 'counter',
 'crafts',
 'crime',
 'criticism',
 'culture',
 'cyberpunk',
 'db',
 'design',
 'det',
 'detective',
 'dict',
 'diy',
 'dramaturgy',
 'east',
 'economics',
 'economy',
 'education',
 'encyc',
 'entertain',
 'epic',
 'epistolary',
 'equ',
 'erotica',
 'esoterics',
 'espionage',
 'etc',
 'european',
 'fairy',
 'family',
 'fanfiction',
 'fantasy',
 'fiction',
 'fight',
 'finance',
 'folklore',
 'foreign',
 'game',
 'garden',
 'geo',
 'geography',
 'gothic',
 'great',
 'guide',
 'guides',
 'hard',
 'health',
 'he

### основные жанры (для дополнительной функции)

In [5]:
gen = {'экшн':'action',
       "литРПГ":'litrpg',
       'приключения':'adventure',
       "архитектура":'architecture',
       "биография":'biography',
       "детское":'child',
       "научная фантастика":'sci sf',
       "классическое":'classic classical',
       "комедия":'comedy',
       "преступления":'crime',
       "фантастика":'fiction fantasy',
       "киберпанк":'cyberpunk',
       "детектив":'det detective',
       "драма":'dramaturgy',
       "сказочное":'fairy',
       "фольколёр":'folklore',
       "ужас":'horror',
       "философия":'philosophy',
       "постапокалипсис":'postapocalyptic',
       "проза":'prose',
       "роман":'romance roman',
       "русское":'russia',
       "сатира":'satire',
       "наука":'science',
       "космос":'space',
       "социальное":'social',
       "поэзия":'poetry',
       'животные':'animal'
       }

In [6]:
with open('genr_list.pkl', 'wb') as fw:
        pickle.dump(gen, fw)

In [7]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,annotation,author,genres_x,title,id,year_y,freq,full_text
0,0,0,0,мнению профессора элленберга математика это на...,Джордан Элленберг,popular math sci,как не ошибаться,9518,2014,20,наука элленберга осмысливать осознаем считавше...
1,1,1,1,итак добро пожаловать полный набор всё начала ...,Милослав Князев,sf fantasy,великая миссия,151215,2011,34,ушастые правка эльфийки круче великого использ...
2,2,2,2,айове стояло безоблачное весеннее утро преддве...,Рэй Брэдбери,prose contemporary,однаединственная ночь,105472,1988,12,заигравшись беспечально твеновский босоногий с...
3,3,3,3,вампир принцессе товарищ верно вампир принцесс...,Кира Александровна Филиппова,sf humor,от принцесс добра не ищут,240262,2008,23,вампир пускается принцессе представительным ко...
4,4,4,4,жизни екатерины миримановой автора системы мин...,Екатерина Валерьевна Мириманова,home sex sci psychology,мужчина и женщина минус 60 проблем в отношениях,273127,2012,40,системы отношения задумалась гармоничные мирим...


### Поиск по названию ближайшей книги

In [10]:
def _clean(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\_', ' ', text)
    return text

In [11]:
def find(name):
    name = set(_clean(name).split())
    best = [0, 0]
    for i in range(len(df)):
        temp = set(df['title'][i].lower().split())
        if temp == name:
            return i
        elif len(name.intersection(temp)) > best[0]:
            best[0] = len(name.intersection(temp))
            best[1] = i
    return best[1]

### Модель

In [12]:
from gensim.models import Word2Vec

В датасете есть annotation (полное описание) и full_text (выбранные с помощью Tfidf.feature_extraction). 
Обучение на annotation работает лучше

In [13]:
data  = list(df['full_text'])

In [14]:
for i in range(len(data)):
    data[i] = data[i].split()

In [16]:
w2v_model = Word2Vec(min_count=5,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=4)

In [17]:
w2v_model.build_vocab(data)

In [18]:
w2v_model.train(data, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(2005607, 8774100)

In [12]:
w2v_model.save('descr_w2v.model')

In [15]:
w2v_model = Word2Vec.load('descr_w2v.model')

In [16]:
gv = CountVectorizer()
X = gv.fit_transform(df['genres_x'])

In [18]:
with open('genres_vec.pkl', 'wb') as fw:
        pickle.dump(gv.vocabulary_, fw)

In [17]:
X = np.array(X.todense())

### Векторизация

In [20]:
def to_vec(mod,sent,gv,genre):
    sent = sent.split()
    v = np.zeros((300,))
    for word in sent:
        if word in mod.wv:
            v+=mod.wv[word]
    v/=len(sent)
    gen = np.array(gv.transform([genre]).todense())[0]
    v = np.hstack((v, gen))
    return v

In [23]:
anwv = AnnoyIndex(300+len(gv.vocabulary_), 'angular')
annots = df['full_text']
genres = df['genres_x']
for i in range(len(annots)):     
    anwv.add_item(i, to_vec(w2v_model,annots[i], gv,genres[i] ))
anwv.build(10)
#anwv.save('descr.ann')

OSError: Invalid argument

In [18]:
anwv = AnnoyIndex(300+len(gv.vocabulary_), 'angular')
anwv.load('descr.ann')

True

### Тестирование

In [21]:
nm = 'Гари Поттер'
ind = find(nm)
av = to_vec(w2v_model, df['full_text'][ind], gv,df['genres_x'][ind])
res = anwv.get_nns_by_vector(av, 10)


In [22]:
df['title'][res]

2817     гарри поттер и дары смерти
8571                медная перчатка
16556                      дом аида
2752                  тени прошлого
5602                 бронзовый ключ
1611                      центурион
7083               золотое снадобье
7339         одд и ледяные великаны
5165                    власть огня
12113                       трианон
Name: title, dtype: object